In [ ]:
import pandas as pd

df = pd.read_csv('eval_parsed_chunks.csv')

In [ ]:
from src.agents_constants import LLAMA3_70_AGENT
from src.agents.message import SystemMessage, UserMessage

agent = LLAMA3_70_AGENT
conf = SystemMessage(
    "You are a Function calling agent, your task is to generate question and answer. Analyze user's text and generate one question and an answer to that question, according to the given data. Call provided function 'generate_qa', provide the question and answer as parameters.")


def get_qa(text):
    def generate_qa(question: str, answer: str):
        """Generate question and answer to the question using given data and call this function"""
        return question, answer

    usr = UserMessage(f"""This is the text entity to process: '{text}'""")
    return agent.get_forced_function_call(locals(), generate_qa, 2, [conf, usr])

In [ ]:
for i, row in df.iterrows():
    text = row['chunk']
    q, a = get_qa(text)
    df.loc[i, 'question'] = q
    df.loc[i, 'answer'] = a

In [ ]:
display(df)

In [ ]:
df.to_csv('qa.csv')

In [5]:
import pandas as pd
from json_repair import repair_json
import json

df = pd.read_csv('qa.csv')
for i, row in df.iterrows():
    d = json.loads(repair_json(row['chunk']))
    print(d)
    df.loc[i, 'type'] = 2 if d['record_type'] == 'event' else 1
    df.loc[i, 'header'] = d['header']

In [6]:
display(df)
df.to_csv('qa.csv')